In [17]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn.functional as F
import cv2
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from pytorchcv.model_provider import get_model as ptcv_get_model # model
import sys, os
import easydict
import torch.backends.cudnn as cudnn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR100
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torch.optim as optim
import matplotlib.pyplot as plt
#%matplotlib inline 

# enable cuda devices
import os    
os.environ['KMP_DUPLICATE_LIB_OK']='True'

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from meta_neural_network_architectures import VGGReLUNormNetwork, ResNet12
from utils.parser_utils import get_args
from data import MetaLearningSystemDataLoader
from experiment_builder import ExperimentBuilder
from utils.basic import show_batch
from few_shot_learning_system import MAMLFewShotClassifier
import prompters
import torchvision.transforms as T

from data_augmentation import mixup_data, cutmix_data, random_flip, random_flip_like_torchvision
from utils.grad_cam import visualize_and_save_grad_cam, make_folder

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
datasets = "mini_imagenet"
# datasets = "tiered_imagenet"
# datasets = "CIFAR_FS"
# datasets = "CUB"

os.environ['DATASET_DIR'] = 'C:/Users/JM/PycharmProjects/MAML/datasets'
# os.environ['TEST_DATASET'] = "tiered_imagenet" # https://mtl.yyliu.net/download/Lmzjm9tX.html
# os.environ['TEST_DATASET'] = "CIFAR_FS" # https://drive.google.com/file/d/1pTsCCMDj45kzFYgrnO67BWVbKs48Q3NI/view
# os.environ['TEST_DATASET'] = "CUB" # https://data.caltech.edu/records/65de6-vp158

In [19]:
os.environ['DATASET_DIR'] = 'C:/Users/JM/PycharmProjects/MAML/datasets'
print("os.environ['DATASET_DIR'] === ", os.environ['DATASET_DIR'])

args = easydict.EasyDict(
{
  "batch_size":2,
  "image_height":84,
  "image_width":84,
  "image_channels":3,
  "gpu_to_use":0,
  "num_dataprovider_workers":4,
  "max_models_to_save":5,
  "dataset_name":"mini_imagenet_full_size",
  "dataset_path":"mini_imagenet_full_size",
  "reset_stored_paths":False,
  "experiment_name":"../MAML_5way_5shot_filter128_miniImagenet",
  "train_seed": 0, "val_seed": 0,
  "indexes_of_folders_indicating_class": [-3, -2],
  "sets_are_pre_split": True,
  "train_val_test_split": [0.64, 0.16, 0.20],
  "evaluate_on_test_set_only": False,

  "total_epochs": 100,
  "total_iter_per_epoch":500, "continue_from_epoch": -2,
  "num_evaluation_tasks":600,
  "multi_step_loss_num_epochs": 15,
  "minimum_per_task_contribution": 0.01,
  "learnable_per_layer_per_step_inner_loop_learning_rate": False,
  "enable_inner_loop_optimizable_bn_params": False,
  "evalute_on_test_set_only": False,

  "max_pooling": True,
  "per_step_bn_statistics": False,
  "learnable_batch_norm_momentum": False,
  "load_into_memory": False,
  "init_inner_loop_learning_rate": 0.01,
  "init_inner_loop_weight_decay": 0.0005,
  "learnable_bn_gamma": True,
  "learnable_bn_beta": True,

  "dropout_rate_value":0.0,
  "min_learning_rate":0.001,
  "meta_learning_rate":0.001,   "total_epochs_before_pause": 100,
  "first_order_to_second_order_epoch":-1,
  "weight_decay": 0.0,

  "norm_layer":"batch_norm",
  "cnn_num_filters":128,
  "num_stages":4,
  "conv_padding": True,
  "number_of_training_steps_per_iter":5,
  "number_of_evaluation_steps_per_iter":5,
  "cnn_blocks_per_stage":1,
  "num_classes_per_set":5,
  "num_samples_per_class":5,
  "num_target_samples": 15,
  "samples_per_iter" : 1,

  "second_order": True,
  "use_multi_step_loss_optimization":False,
  "backbone": "4-CONV",
  "use_bias": True,
  "prompter": False,
  "prompt_engineering": 'not',
  "prompt_size" : 5,
  "image_size" : 84,
  "prompt_random_init": False,
  "outer_prompt_learning_rate": 0.001,
  "inner_prompt_learning_rate": 0.01,
  "ANIL": False,
  "BOIL": False,
  "data_aug" : "none",
  "ablation_record": False,
  "random_prompt_init": False
}
)

device = torch.cuda.current_device()
args.im_shape = (2, 3, args.image_height, args.image_width)

args.use_cuda = torch.cuda.is_available()
args.seed = 104
args.reverse_channels=False
args.labels_as_int=False
args.reset_stored_filepaths=False
args.num_of_gpus=1

args.continue_from_epoch='latest'

os.environ['DATASET_DIR'] ===  C:/Users/JM/PycharmProjects/MAML/datasets


In [20]:
# 모델을 구성한다
model = MAMLFewShotClassifier(args=args, device=device,
                              im_shape=(2, 3,
                                        args.image_height, args.image_width))

data = MetaLearningSystemDataLoader

maml_system = ExperimentBuilder(model=model, data=data, args=args, device=device)
maml_system.state

Using max pooling
meta network params
Inner Loop parameters
Outer Loop parameters
classifier.layer_dict.conv0.conv.weight torch.Size([128, 3, 3, 3]) cuda:0 True
classifier.layer_dict.conv0.conv.bias torch.Size([128]) cuda:0 True
classifier.layer_dict.conv0.norm_layer.bias torch.Size([128]) cuda:0 True
classifier.layer_dict.conv0.norm_layer.weight torch.Size([128]) cuda:0 True
classifier.layer_dict.conv1.conv.weight torch.Size([128, 128, 3, 3]) cuda:0 True
classifier.layer_dict.conv1.conv.bias torch.Size([128]) cuda:0 True
classifier.layer_dict.conv1.norm_layer.bias torch.Size([128]) cuda:0 True
classifier.layer_dict.conv1.norm_layer.weight torch.Size([128]) cuda:0 True
classifier.layer_dict.conv2.conv.weight torch.Size([128, 128, 3, 3]) cuda:0 True
classifier.layer_dict.conv2.conv.bias torch.Size([128]) cuda:0 True
classifier.layer_dict.conv2.norm_layer.bias torch.Size([128]) cuda:0 True
classifier.layer_dict.conv2.norm_layer.weight torch.Size([128]) cuda:0 True
classifier.layer_dict.c

{'best_val_acc': 0.6124888875087102,
 'best_val_iter': 49000,
 'current_iter': 50000,
 'best_epoch': 98,
 'train_loss_mean': 0.7120679829120636,
 'train_loss_std': 0.14076044237870394,
 'train_accuracy_mean': 0.7360666686296463,
 'train_accuracy_std': 0.06549975402460093,
 'train_loss_importance_vector_0_mean': 0.006000000052154064,
 'train_loss_importance_vector_0_std': 0.0,
 'train_loss_importance_vector_1_mean': 0.006000000052154064,
 'train_loss_importance_vector_1_std': 0.0,
 'train_loss_importance_vector_2_mean': 0.006000000052154064,
 'train_loss_importance_vector_2_std': 0.0,
 'train_loss_importance_vector_3_mean': 0.006000000052154064,
 'train_loss_importance_vector_3_std': 0.0,
 'train_loss_importance_vector_4_mean': 0.9760000109672546,
 'train_loss_importance_vector_4_std': 0.0,
 'train_learning_rate_mean': 0.0010000000000000005,
 'train_learning_rate_std': 4.336808689942018e-19,
 'val_loss_mean': 1.0103712218999863,
 'val_loss_std': 0.14254508195556043,
 'val_accuracy_mean'

In [21]:
model_save_dir = maml_system.saved_models_filepath
model_name = "train_model"
model_idx = maml_system.state['best_epoch']

state = maml_system.model.load_model(model_save_dir=model_save_dir,
                                     model_name=model_name,
                                     model_idx=model_idx+1)

state_dict_loaded = state['network']

maml_system.model.load_state_dict(state_dict=state_dict_loaded)

# # 잘 불러왔는지 확인하는 코드
# print("state_dict_loaded == ",state_dict_loaded)
# print("="*10)
# for key, value in maml_system.model.named_parameters():
#     print(key)
#     print(value)
maml_system.model

MAMLFewShotClassifier(
  (classifier): VGGReLUNormNetwork(
    (layer_dict): ModuleDict(
      (conv0): MetaConvNormLayerReLU(
        (layer_dict): ModuleDict()
        (conv): MetaConv2dLayer()
        (norm_layer): MetaBatchNormLayer(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (conv1): MetaConvNormLayerReLU(
        (layer_dict): ModuleDict()
        (conv): MetaConv2dLayer()
        (norm_layer): MetaBatchNormLayer(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (conv2): MetaConvNormLayerReLU(
        (layer_dict): ModuleDict()
        (conv): MetaConv2dLayer()
        (norm_layer): MetaBatchNormLayer(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (conv3): MetaConvNormLayerReLU(
        (layer_dict): ModuleDict()
        (conv): MetaConv2dLayer()
        (norm_layer): MetaBatchNormLayer(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (line

In [22]:
train_data = maml_system.data.get_train_batches(total_batches=int(600/2), augment_images=False)

figure_idx = 0

for sample_idx, train_sample in enumerate(train_data):
    
    x_support_set, x_target_set, y_support_set, y_target_set, seed = train_sample
    
    x_support_set = torch.Tensor(x_support_set).float().to(device=maml_system.model.device)
    x_target_set = torch.Tensor(x_target_set).float().to(device=maml_system.model.device)
    y_support_set = torch.Tensor(y_support_set).long().to(device=maml_system.model.device)
    y_target_set = torch.Tensor(y_target_set).long().to(device=maml_system.model.device)
    
    for task_id, (x_support_set_task, y_support_set_task, x_target_set_task, y_target_set_task) in enumerate(zip(x_support_set,
                              y_support_set,
                              x_target_set,
                              y_target_set)):
        
        names_weights_copy = maml_system.model.get_inner_loop_parameter_dict(maml_system.model.classifier.named_parameters())
        
        num_devices = torch.cuda.device_count() if torch.cuda.is_available() else 1

        names_weights_copy = {
            name.replace('module.', ''): value.unsqueeze(0).repeat(
                [num_devices] + [1 for i in range(len(value.shape))]) for
            name, value in names_weights_copy.items()}
        
        prompted_weights_copy = {}
        if args.prompter:
            prompted_weights_copy = {key: value for key, value in names_weights_copy.items() if 'prompt' in key}

        names_weights_copy = {key: value for key, value in names_weights_copy.items() if 'layer_dict' in key}
        
        n, s, c, h, w = x_target_set_task.shape

        x_support_set_task = x_support_set_task.view(-1, c, h, w)
        y_support_set_task = y_support_set_task.view(-1)
        x_target_set_task = x_target_set_task.view(-1, c, h, w)
        y_target_set_task = y_target_set_task.view(-1)
        
        num_steps=5
        
        for num_step in range(num_steps):            
            
            support_loss, support_preds = maml_system.model.net_forward(
                                                               x=x_support_set_task,
                                                               y=y_support_set_task,
                                                               weights=names_weights_copy,
                                                               prompted_weights=prompted_weights_copy,
                                                               backup_running_statistics=num_step == 0,
                                                               prepend_prompt=args.prompter,
                                                               training=True,
                                                               num_step=num_step,
                                                               training_phase=False,
                                                               epoch=0,
                                                               inner_loop=True)
        
            names_weights_copy, prompted_weights_copy = maml_system.model.apply_inner_loop_update(
                 loss=support_loss,
                 names_weights_copy=names_weights_copy,
                 prompted_weights_copy=prompted_weights_copy,
                 use_second_order=True,
                 current_step_idx=num_step,
                 current_iter='test',
                 training_phase=False)
            
        ##########Inner level 종료 #############
        # Query set으로 Grad-CAM을 구한다
        individual_images = torch.split(x_target_set_task, 1, dim=0)  # (25, 3, 84, 84) -> (1, 3, 84, 84) 텐서로 분리
        individual_labels = torch.split(y_target_set_task, 1, dim=0)
        
        
        original_save_path = "Grad_CAM/" + datasets + "/MAML/origin/" + "sample_idx_" + str(sample_idx) + "/" + "task_id_" + str(task_id) + "/"
        grad_cam_save_path = "Grad_CAM/" + datasets + "/MAML/gradcam/"+ "sample_idx_" + str(sample_idx) + "/" + "task_id_" + str(task_id) + "/"
        
        make_folder(original_save_path)
        make_folder(grad_cam_save_path)
        
        # 각 이미지 텐서 확인
        for i in range(y_target_set_task.shape[0]):

            input_image = individual_images[i]
            y_label = individual_labels[i]

            _, target_preds, feature_map_list = maml_system.model.net_forward_feature_extractor(x=input_image,
                                                             y=y_label, weights=names_weights_copy,
                                                             backup_running_statistics=False, training=True,
                                                             num_step=num_step, training_phase='test',
                                                             epoch=0)
            
            feature_map = feature_map_list[3]
            
            # 선택한 클래스에 대한 스칼라 출력값을 기준으로 gradient 계산
            target_class = y_label
            output = target_preds

            class_score = output[:, target_class].sum()
            gradients = torch.autograd.grad(outputs=class_score, inputs=feature_map, grad_outputs=torch.ones_like(class_score))

            # gradients를 사용하여 feature map의 가중치를 계산
            weights = torch.mean(gradients[0], dim=(2, 3), keepdim=True)
            grad_cam = F.relu(torch.sum(weights * feature_map, dim=1, keepdim=True))

            # grad_cam을 이미지 크기로 리사이즈
            grad_cam = F.interpolate(grad_cam, size=(input_image.shape[2], input_image.shape[3]), mode='bilinear', align_corners=False)
            grad_cam = grad_cam.squeeze().cpu().detach().numpy()

            visualize_and_save_grad_cam(
                input_image=input_image,
                grad_cam=grad_cam,
                grad_cam_save_path=grad_cam_save_path + str(i) + ".png",
                original_save_path=original_save_path + str(i) + ".png",
                datasets=datasets)